In [106]:
import os
import sys, os
import dill
import sqlalchemy
import traceback
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from aepsych.server.sockets import PySocket
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text
from IPython.core.display import HTML as html
import pandas as pd
import numpy as np

test = widgets.Output(layout=Layout(border="2px solid blue"))
display(test)


connect_out = widgets.Output(layout=Layout(padding="2px", height="150px"))
!true > logs/bayes_opt_server.log #  clears the server.log file data when you start

with connect_out:
    server = None
    connect_out.clear_output()
strat = None
db_file_name = None
db_config_str = None
database_path = None
new_experiment = False
resume_experiment = False
dim = 0


#---------- Style ----------

display(html("""<style>
.jp-CodeCell {
    width:100% !important;
    background: whitesmoke;
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

.p-Collapse-header,
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current:before,
.jupyter-button ,
.widget-button,
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border: 1px solid #dcdcdc;
}


.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab{
border-top-right-radius:4px;
border-top-left-radius: 4px;
border: 1px solid #dcdcdc;
}
.p-Collapse-contents{
border-bottom-right-radius:4px;
border-bottom-left-radius: 4px;
border: 1px solid #dcdcdc;
border-top:none;
}

.p-Collapse-open > .p-Collapse-header {
border-bottom-right-radius:0px;
border-bottom-left-radius: 0px;
}
body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
   # padding: 10px !important;
   # background: white;
   # border-radius: 4px !important;
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # border: 1px solid #e3e3e3;
   # border-radius: 4px;
   # -webkit-box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # margin: 10px;
   # max-width: 600px; 
    height: auto !important;
    width: auto !important;
   # overflow-y: auto;
}

.jp-OutputArea {
    overflow-y: inherit;
}

.widget-text input[type="text"],
.widget-text input[type="number"],
.widget-dropdown > select {
   border-radius: 4px;
   box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}

.widget-vbox {
    justify-content: center;
    width: 100%;
    width: -webkit-fill-available !important;
    display: flex;
    border-radius: 4px;
    background: white;
    height: fit-content;
    padding: 20px;
   # box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
}


[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}


.jupyter-widgets.widget-tab {
    min-width: 30% !important;
}
html {
    background: #F2F2F2;
    height: 100%;
    margin: 0px;
    padding: 0px;

}

.widget-dropdown,
.widget-text{
    width: auto;

}
.widget-label {
    display: flex;
    align-items: center;
}
.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {
    background: white;
    padding-top: var(--jp-code-padding);
}
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border-top-left-radius: 0px;
    border-top-right-radius: 0px;
}
.widget-textarea textarea {
    height: auto;
    width: 600px;
    min-height: 100%;
    resize: none;
    background-color:black; 
    color: green;
}
.widget-box {
    overflow: hidden !important;
}
.jp-OutputArea-output {
    overflow: hidden
    }
.jp-RenderedHTMLCommon p {
    margin-bottom: 0;
}
div.output_stderr {
    background: #fdd;
    display: none;
}

.connect_container_color {
background-color: #F5F5F5;
margin-top: 10px;
}
</style>"""))


header = HTML(
    """
<div class="nav" style=" 
margin: 0px;
background-color: #F2F2F2;
font-weight: bold; 
box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
">
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    align-items: center; 
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color:rgb(80,103,132)">
    <img src="./logo.png" alt="logo" width="50" height="50" style="margin-right: 10px;" >
    AEPsych Visualizer
    </h1>
</div>
"""
)

logs_style = Layout(width='600px', height='400px')
input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="10px",width="95%")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    layout={
            "overflow": "scroll", 
            "height": "50vh",
            "width": "100%",
            "padding": "5px"
           }
)

# ---------- Inputs -----------

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]   


uploader = widgets.FileUpload(
    description="Select File",accept=".db", multiple=False, layout=btn_style)

yes_label = Text(
    value="Yes Trial", placeholder="yes_label",
    description="yes_label: ", disabled=False, style=input_style,
    layout=input_layout
)

no_label = Text(
    value="No Trial", placeholder="no_label", 
    description="no_label: ", disabled=False, style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")


# -------- ports ----------
server_ip = widgets.Text(
    placeholder="0.0.0.0", # must use regex to check pattern and type int
    description='Server IP',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px"))
port = widgets.Text(
    placeholder="5555", # must add regex, type check, and error handling
    description='Port',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px", justify_content="center"))

connected_ip = widgets.Output(ayout=Layout(display="flex", justify_content="flex-end"))

def on_change_server_input(change):
    with inputs_output:
        change["new"]

                              
server_ip.observe(on_change_server_input, names="value")
port.observe(on_change_server_input, names="value") 

button_submit_port_ip = widgets.Button(
    description="Start Server",
    disabled=False,
    button_style="info",
    tooltip="submit",
    layout=Layout(align_self="flex-end", width="fit-content", margin="10px"),
)


def mv_db_file(server_ip):
    mv_path = f"!mv {server_ip} /Users/ecortez/Work/voila/notebooks/databases" 
    return mv_path


def submit_port_ip_clicked(b):
    # This is for new experiments
    global server
    connected_ip.clear_output()
    try:
        if len(port.value) and len(server_ip.value):
            server = AEPsychServer(port.value, server_ip.value)
            print(mv_db_file(server_ip.value))
#             mv_path = f"!mv {server_ip.value} " + os.getcwd()+"/databases"
#             mv_path
 
            
            with connected_ip:
                print(f"Connected by Client at {server_ip.value}")
        else:
            print("Server is connected to default port and IP address. Enter a port or ip address.")
    except Exception as e:
        print(f"Unable to connect at ip: {server_ip.value} and port {port.value}")
        print(e)


button_submit_port_ip.on_click(submit_port_ip_clicked)
connection_input_box = widgets.Box(
    [server_ip, port, button_submit_port_ip], layout=Layout(justify_content="flex-end", margin="4px")
)
connection_inputs = widgets.VBox([connection_input_box, connected_ip], 
                                 layout=Layout(min_height="15px", display="flex", justify_content="flex-end"))

connection_inputs.add_class("connect_container_color")
# -------- Config --------
    
# New Config

config_str = """
[common]
parnames = [theta, test] # names of the parameters
lb = [0.1, 0.1] # lower bound of the parameter
ub = [3, 5] # upper bound of parameter
stimuli_per_trial = 1 # the number of stimuli shown in each trial; 1 for single, or 2 for pairwise experiments
outcome_types = [binary] # the type of response given by the participant; can be [binary] or [continuous]
target = 0.75 # desired threshold, for threshold estimation.
strategy_names = [init_strat, opt_strat] # The strategies that will be used, corresponding to the named sections below

# Configuration for the initialization strategy, which we use to gather initial points
# before we start doing model-based acquisition
[init_strat]
min_total_tells = 5 # number of sobol trials to run
generator = SobolGenerator # The generator class used to generate new parameter values

# Configuration for the optimization strategy, our model-based acquisition
[opt_strat]
min_total_tells = 50 # total number of trials to run
refit_every = 5 # how often to refit the model from scratch
generator = OptimizeAcqfGenerator # The generator class used to generate new parameter values
acqf = MCLevelSetEstimation # The acquisition function; MCLevelSetEstimation is used for threshold finding
model = GPClassificationModel # The model class"""

config_output = widgets.Output()

config_textarea = widgets.Textarea(value=config_str, layout=Layout(width="100%", height="400px"))
config_textarea.observe(on_value_change_input, names="value")

button_submit_config = widgets.Button(
    description="Save Config File",
    disabled=False,
    button_style="info",
    tooltip="submit",
    # layout=btn_style
)
button_reset_config = widgets.Button(
    description="reset",
    disabled=False,
    button_style="",
    tooltip="submit",
    # layout=btn_style
)


def make_config():
    global server
    config = config_textarea.value
    
    try:
        config_output.clear_output()
        with config_output:
            
            if server is not None:
                server.configure(config_str=config)
                print("Config sent to server successfuly")
            else:
                print("Server not connected")
    except Exception as e:
        print("Unable to configure server. Please reset and reconfigure.")
        print(e)


def clicked_submit_config(b):
    make_config()


def clicked_reset_config(b):
    config_output.clear_output()
    config_textarea.value = config_str


button_submit_config.on_click(clicked_submit_config)
button_reset_config.on_click(clicked_reset_config)


def config_input(change):
    change.new


config_textarea.observe(config_input, names=["value"])
config_buttons = widgets.HBox(
    [button_submit_config, button_reset_config],
    layout=Layout(display="flex", justify_content="flex-end", marging="5px 0"),
)


# Resume Config
default_config_file = None


config_output = widgets.Output(layout=Layout(margin="10px"))

def update_config_value():
    global db_config_str
    config_output.clear_output()
    with config_output:
        display(html(f"""
        <div style='height: 400px;'>
            <pre
            style='font-family: monospace;background-color: black;
            color: green;padding: 8px;height: 100%;overflow: auto; '
            >{db_config_str}</pre>

        </div>
        """))


def show_config():
    '''
    Displays database config. (View only)
    '''
    global server
    config = config_textarea.value
    config_output.clear_output()
#     try: 
#         with config_output:
            
# #             server.configure(config_str=config) # New confgi
# #             print("Submit, need to add functionality to update config in db?")
#     except Exception as e:
# #             print("Unable to configure server. Please reset and reconfigure.")
#             print(e)

                
# def clicked_submit_config(b):
#     show_config()
    
    

# def clicked_reset_config(b):
#     global default_config_file 
#     config_output.clear_output()
#     config_textarea.value = default_config_file
    
    
# button_submit_config.on_click(clicked_submit_config)
# button_reset_config.on_click(clicked_reset_config)

# -------- Plot view ----------

# --------- Accordion -----------
button_submit = widgets.Button(
    description="Submit", disabled=False, 
    button_style="info", tooltip="submit", 
    layout=btn_style
)
accordion = widgets.Accordion(
    children=[
        widgets.VBox([no_label, yes_label], layout=Layout(padding="5px")),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ],
            layout=Layout(padding="5px")
        ),
        widgets.VBox([xlabel, ylabel],layout=Layout(padding="5px")),
    ],
    layout=Layout(min_width="280px")
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")




def submit():
    file_output.clear_output()
    with file_output:
        display_plot()
        
        
def submit_clicked(b):
    submit()

    
button_submit.on_click(submit_clicked)

plot_interact_box = widgets.VBox([accordion, button_submit])

# -------- Logging --------
import logging
import aepsych.utils_logging as utils_logging

logger_output = widgets.Output()

log_path = os.getcwd() + "/logs/bayes_opt_server.log"


# def load_logs():
#     logger_output.clear_output()
#     with logger_output:
#         with open(log_path) as f:
#             contents = f.readlines()
#             for log in contents:
#                 print(log)

def load_logs():
    logger_output.clear_output()
    log_str = ""
    logger_output.clear_output()
    with logger_output:
        with open(log_path) as f:
            contents = f.readlines()
            for log in contents:
                log_str += f"<p style='margin-top: 1px;margin-bottem: 1px'>{log}</p>"
        display(html(f"""
        <div style='height: 400px;overflow: auto;'>
            <pre
            style='font-family: monospace; background-color: black; color: white; 
            padding: 4px; height: 100%;'
            >{log_str}</pre>
        </div>
        """))
                

# --------- Interactive ------------

param_output = widgets.Output(
    layout=Layout(
        padding="2px", height="auto", align_items="center", justify_conten="center", min_width="400px", width="auto"
    )
)

# param_one = widgets.FloatText(value=0, description="Parameter1: ", disabled=False, layout=Layout(margin="5px 10px"))
# param_two = widgets.FloatText(value=0, description="Parameter2: ", disabled=False, layout=Layout(margin="5px 10px"))
outcome = widgets.IntText(value=0, description="Outcome: ", disabled=False, layout=Layout(margin="5px 10px"))
# param_one, param_two, outcome
param_box_tab = widgets.VBox([], layout=Layout(max_width="350px")) 



interactive_btn_labels = ['New Parameters', 'Update Model', 'Send to Client']

interactive_buttons = [
    widgets.Button(description=f"{interactive_btn_labels[i]}",
    disabled=False,
    button_style='',
    tooltip='Click me',) for i in range(3)]

inteact_btn_box = widgets.Box(interactive_buttons)
interactive_buttons[2].disabled = True

interactive_container = widgets.VBox([inteact_btn_box, param_box_tab, param_output])


def get_next(d):  # ask
    global server
    global trial_params
    global strat
    global db_config_str
    param_output.clear_output()
    ask_message = {"type": "ask", "version": "0.01", "message": ""}
    ask_response = server.versioned_handler(ask_message)["config"]
    param_output.clear_output()
    with param_output:
        children = []
        for name, data in ask_response.items():
            children.append(widgets.FloatText(value=data[0], description=f"{name}", disabled=False, layout=Layout(margin="5px 10px")))
        
        children.append(outcome)
        param_box_tab.children = children
        
    
#         display(
#             html(
#                 f"""
#         <div style='height:400px;overflow:auto'>
#             <table style='width: 100%;'>
#                 <caption>Next Parameter</caption>
#                 <tr>
#                     {param_th_str}
#                 </tr>
#                 {param_td_str}
#             </table>
#         </div>
#         """
#             )
#         )




def tell_model(d):  # Tell
    dim = len(server._parnames[0])
    param_output.clear_output()
    
    tell_msg = {}
    for param in param_box_tab.children: 
        tell_msg[param.description] = param.value
    
    outcome_val = outcome.value
           
#     if dim > 0 and dim < 3:
#         param1 = param_one.value
#     if dim == 2:
#         param2 = param_two.value
#     outcome_val = outcome.value

#     tell_message = {}
#     if dim > 0 and dim < 3:
#         tell_message[server._parnames[0][0]] = param1
#     if dim == 2:
#         tell_message[server._parnames[0][1]] = param2
    try:
        message = {
            "type": "tell",
            "version": "0.01",
            "message": {
                "config": tell_msg,
                "outcome": outcome_val,
            },
        }
        server.versioned_handler(message)
        with param_output:
            print("Received msg [tell]")
        for child in param_box_tab.children: 
            child.value = 0
    except Exception as e:
        with param_output:
            print("Unable to update Model...")
            print(e)
    
   
    # if server.strat.x is not None:
    #     data = {par: server.strat.x[:, i] for i, par in enumerate(server.parnames)}
    #     data["outcome"] = server.strat.y
    #     data = pd.DataFrame(data)
    #     #  param_output.clear_output()
    #     display(
    #         html(
    #             f"""<div style='height: 200px; overflow: auto; width: 100%'>
    #      <p style="margin: 1px;">Server Data</p>
    #     {data.style.render()}
    #     </div>"""
    #         )
    #     )


# This will just be to send a message to the server
def send_to_client(d):
    param_output.clear_output()
    with param_output:
        print(
            """Send to Client ----> To Do 
        #  sent the outcome info to the server so that it can update its model 
        # server.tell(outcome, trial_params)
        """
        )


interactive_buttons[0].on_click(get_next)
interactive_buttons[1].on_click(tell_model)
interactive_buttons[2].on_click(send_to_client)

# trial_params = None

# def new_parameters_clicked(d):    #ask 
#     global server
#     global trial_params
    
#     param_output.clear_output()
#     with param_output:
# #         trial_params = server.ask()
#         print("""new_parameters clicked! ---> in progress 
#         #ask the server what parameter values to try next 
#         # server.ask() 
#         """)
#     load_logs()


# def update_model_clicked(d):   # Tell 
#     param_output.clear_output()
#     with param_output: 
#         print("""Update Model ----> To Do  
#         # tell the server the outcome 
#         # store the outcome in a variable that a user can update via an input
#         """)


# def send_to_client(d):
#     param_output.clear_output()
#     with param_output: 
#         print("""Send to Client ----> To Do 
#         #  sent the outcome info to the server so that it can update its model 
#         # server.tell(outcome, trial_params)
#         """)

# interactive_buttons[0].on_click(new_parameters_clicked)
# interactive_buttons[1].on_click(update_model_clicked)
# interactive_buttons[2].on_click(send_to_client) 
# ------- View 2 --------
# --> plot view 
plot_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))

# --> table view


table_data_output = widgets.Output(layout=Layout(width='600px',height='400px',))



######### DOWNLOAD SECTION ###################
from tempfile import NamedTemporaryFile, TemporaryDirectory
from zipfile import ZipFile
            
#     for root, dir, files in os.walk("."): 
#         abs_root = os.path.abspath(root)
#         print(abs_root)
            

default_db_file_path = os.getcwd() + '/databases/default.db'
# print("full path", default_db_file_path)


# https://stackoverflow.com/questions/72361140/how-can-i-make-a-temporaty-zip-object-downlaodable-for-user-using-jupyter-notebo
    
def download_link():
#     global default_db_file_path
#     download_link_tag = html(f"<a href=f'{default_db_file_path }' type='application/vnd.sqlite3' download='default.db' >Download</a>")
    table_data_output.clear_output() 
    with table_data_output:
#         display(download_link_tag)
#         print("Does path exisit?", os.getcwd()+'/databases/default.db', os.path.isdir(os.getcwd()+'/databases/default.db'))
        
#         with TemporaryDirectory(dir=".") as tmpdir: #make dir
#             print("temp directory", tmpdir)
#             with NamedTemporaryFile(dir=tmpdir)as t: # make file
# #                 t.name += ".zip"
#                 name = t.name
#                 print(name)
                
        
        
                
        title = "Download_db_file"
        filename = "database.db"
        file = default_db_file_path
#         zip_fn = os.path.join(file, filename)
        
        
        

        with ZipFile('database.zip', 'w') as zipObj:
           # Add multiple files to the zip
           zipObj.write('./databases/default.db')
        
        
#         display(html("""
#         <a href="./voila/static/database.zip" download="test.zip">Download the Resulting Files as an Archive</a>
#         """))
            # CREATS ZIP file 
#         !zip -r experiment_database.zip databases/default.db
        
        
        with ZipFile("./database.zip", 'r') as zip:
            # printing all the contents of the zip file
            
            payload = zip
#             print(payload)   
                    #BUTTONS
            html_buttons = '''<html>
            <head>
            <meta name="viewport" content="width=device-width, initial-scale=1">
            </head>
            <body>
            <a download="{filename}" href="data:application/zip;{payload}" download>
            <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
            </a>
            </body>
            </html>
            '''

            html_button = html_buttons.format(payload=payload,filename=filename)
            display(HTML(html_button))




        


######################################


# def display_data():
#     global db_config_str
#     with table_data_output:
#         uuid = server.db.get_master_records()[0].experiment_id
#         strat = server.get_strat_from_replay()
#         replay_data = server.db.get_replay_for(uuid)
#         print('{:<4}{:>20}{:>20}{:>25}'.format("unique_","message_type","master_talbe_id","message_contents"))

#         for record in replay_data:
#             if record.message_type == 'setup':   
#                 db_config_str = record.message_contents['message']['config_str']
#                 update_config_value()
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id, 
#                       'view config under Exp details',)) 
#             else: 
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id,
#                       str(record.message_contents['message'])))
            
    
        
#         data = server.get_dataframe_from_replay()
#         display("data", data)

def display_data():
    global db_config_str
    with table_data_output:
        uuid = server.db.get_master_records()[0].experiment_id
        replay_data = server.db.get_replay_for(uuid)
        setup_table_row = ""
        table_rows = ""

        for record in replay_data:
            if record.message_type == 'setup':
                db_config_str = record.message_contents['message']['config_str']
                setup_table_row += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>view config under Exp details tab</th>
                </tr>
                """
            else: 
                table_rows += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>{str(record.message_contents['message'])}</th>
                </tr>
                """
        display(html(f"""
          <div style='height: 400px; overflow: auto;'>
            <table style='width: 100%;'>
                <tr>
                    <th>unique_</th>
                    <th>message_type</th>
                    <th>master_talbe_id</th>
                    <th>message_contents</th>
                </tr>
                {setup_table_row}
                {table_rows}
            </table>
          </div>
        """))

        
# ---> use for new session 

def display_plot():
    global strat
    plot_output.clear_output()
    with plot_output:
#     #------- Error Handling -----
        error = False
        with plot_output:
            plot_output.clear_output()
            if len(ylabel.value) > 35: 
                error = True
    #             raise Exception("ylabel must be less than 35 characters ")
                print("ylabel must be less than 35 characters ")
            elif len(xlabel.value) > 35: 
                error = True
                print("xlabel must be less than 35 characters ")
            elif len(yes_label.value) > 35: 
                error = True
                print("yes_label must be less than 35 characters ")
            elif len(no_label.value) > 35: 
                error = True
                print("no_label must be less than 35 characters ")
   
        if error == False: 
            inputs_output.clear_output() 
        try: 
            plot_strat(
                server.strat,
                xlabel=xlabel.value,
                ylabel=ylabel.value,
                no_label=no_label.value,
                yes_label=yes_label.value,
                cred_level=cred_level.value,
                target_level=target_level.value,
            )  
        except AttributeError as err:
            print("Collect more data, \n")
            print(err)
            traceback.print_exc()
            
        except AssertionError as err:
            print("Collect more data, \n")
            print(err)
            traceback.print_exc()
        except Exception as e:
            print(e) 
            traceback.print_exc()


out = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="flex-start",
    margin="0",
    width="100%"
))

exp_details = widgets.Button(
    description='Exp Details',
    disabled=False,
    button_style='',
    tooltip='Click me')
plot_view = widgets.Button(
    description="Plot View",
    disable=False,
    button_style="",
    tooltip="Click me")
table_view = widgets.Button(
    description="Table View",
    disable=False,
    button_style="",
    tooltip="Click me")
logs = widgets.Button(
    description="Logs",
    disable=False,
    button_style="",
    tooltip="Click me")
interactive = widgets.Button(
    description="Interactive",
    disable=False,
    button_style="",
    tooltip="Click me")

navigation = widgets.Box([exp_details,plot_view,table_view,logs,interactive],
                         
                         
layout=Layout(  
        display='flex',
        flex_flow="column",
        width='200px',
        margin="0 2.5%",
        min_width="152px"
        ))
output_header_style = Layout(justify_content="center")
nav_display_style = Layout(display="flex",
        flex_flow="column")


container_one = widgets.Box([
    Label("Exp Details", layout=output_header_style),
    config_output],
    layout=nav_display_style )
container_one_new = widgets.Box([config_textarea,config_buttons, config_output],layout=nav_display_style)

container_two = widgets.Box([
    Label("Plot View", layout=output_header_style),
    widgets.HBox([plot_output,plot_interact_box])],
    layout=Layout(width="100%",display="flex",flex_flow="column"))

container_three = widgets.Box([
    Label("Table View"),table_data_output],
    layout=nav_display_style)

container_four = widgets.Box([
    Label("Logs", layout=output_header_style),
    logger_output],
    layout=nav_display_style)

container_five = widgets.Box([interactive_container], layout=Layout(margin="0",padding="0"))

nav_container = widgets.Box(
    [navigation,out],
    layout=widgets.Layout(
    display="flex",
    flex_flow='row'
    ))
main_container = widgets.Box(
    [connection_inputs,nav_container],
    layout=widgets.Layout(
    display="flex",
    flex_flow="column",
    marging="auto"
    )
)

# display(main_container)


def nav_btn_click(b):
    show_display = b.description
    out.clear_output()
    if show_display == 'Exp Details':
        with out:
            if new_experiment: 
                display(container_one_new)
            if resume_experiment:  
                display(container_one)
    elif show_display == "Plot View":
        display_plot() 
        with out:
            display(container_two)
    elif show_display == "Table View":
        download_link()
        with out: 
            display(container_three)
    elif show_display == "Logs":
        with out:
            load_logs()
            display(container_four)
    elif show_display == "Interactive":
#         ask_for_params()
        with out: 
            display(container_five)

            
exp_details.on_click(nav_btn_click)    
plot_view.on_click(nav_btn_click)
table_view.on_click(nav_btn_click)
logs.on_click(nav_btn_click)
interactive.on_click(nav_btn_click)


#------ View 1 ------            
uploader = widgets.FileUpload(
    description="Connect to Database",accept=".db", multiple=False, layout=Layout(margin="15px 60px"))

def start_server():
    global strat
    global server

    with error_output:
        try:
            server = AEPsychServer()
            make_config()
#             download_link()
            config_output.clear_output()
            with out: 
                display(container_one_new)
            # strat = server.get_strat_from_replay()
            # display_data()
            # if display_plot() == True:
            home_view_container.layout.display = "none"
            main_container.layout.display = "inline"
            display(main_container)
        except Exception as e:
            print("Unable to start a new experiment")
            print(e)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
    
    
#     config = show_config()
#     global strat
#     global server
#     with error_output:
#         try:
#             server = AEPsychServer(database_path=database_path)
#             strat = server.get_strat_from_replay()
#             print(lent(strat))
#             display_data()
#             if display_plot() == True:
#                 home_view_container.layout.display = 'none' 
#                 main_container.layout.display = 'inline'
#             else:
#                 error_output.clear_output()
#                 if strat != None:
#                     try:
#                         plot_strat(
#                             strat,
#                             xlabel=xlabel.value,
#                             ylabel=ylabel.value,
#                             no_label=no_label.value,
#                             yes_label=yes_label.value,
#                             cred_level=cred_level.value,
#                             target_level=target_level.value,
#                         )
#                     except RuntimeError as err: 
#                         error_output.clear_output()
#                         with error_output:
#                             print("Unable to connect to server. \n")
#                             print(err)
#                     except AssertionError as err: 
#                         print(err)

#                 display(main_container)
#         except sqlalchemy.exc.DatabaseError as err:
#                     error_output.clear_output()
#                     with error_output:
#                         print("Unable to connect to server. \n")
#                         print(err)
#         except Exception as e:
#             print("unable to connect to server...")
#             print(e)
#     #             exc_type, exc_obj, exc_tb = sys.exc_info()
#     #             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#     #             print(exc_type, fname, exc_tb.tb_lineno)    
    
    
    
def resume_server():
    global strat
    global server
    with error_output:
        try:
            !true > logs/bayes_opt_server.log
            server = AEPsychServer(database_path=database_path)
            strat = server.get_strat_from_replay()
            display_data()
#           clears the server.log file data when you start
#             if display_plot() == True:
            update_config_value()
            with out:
                display(container_one)
            home_view_container.layout.display = 'none' 
            main_container.layout.display = 'inline'
#             else:
#                 error_output.clear_output()
#                 if strat != None:
#                     try:
#                         plot_strat(
#                             strat,
#                             xlabel=xlabel.value,
#                             ylabel=ylabel.value,
#                             no_label=no_label.value,
#                             yes_label=yes_label.value,
#                             cred_level=cred_level.value,
#                             target_level=target_level.value,
#                         )
#                     except RuntimeError as err: 
#                         error_output.clear_output()
#                         with error_output:
#                             print("Unable to connect to server. \n")
#                             print(err)
#                     except AssertionError as err: 
#                         print(err)

#                 display(main_container)
#         except sqlalchemy.exc.DatabaseError as err:
#                     error_output.clear_output()
#                     with error_output:
#                         print("Unable to connect to server. \n")
#                         print(err)
        except Exception as e:
            print("unable to connect to server...")
            print(e)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            

def upload():
    global server
    global strat
    global database_path
    global db_file_name
    error_output.clear_output() 
    with error_output:     
        error_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                db_file_name = file_name 
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    resume_server()
                    f.close()
                    

def uploader_on_change(change): 
    error_output.clear_output()
    with error_output:
        try: 
            upload()
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)

uploader.observe(uploader_on_change, names="value")

error_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
build_experiment = widgets.Button(
    description='Build Experiment',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=Layout(width='auto', height="auto", margin="15px 60px"))

home_view_container = widgets.Box([
    Label("Build a new experiment, or load data from an old one"), 
    widgets.HBox([build_experiment, uploader], 
                layout=widgets.Layout(
                display="flex",
                justify_content="center",
                margin="30px 0",
                align_items="center"
                )),
    error_output,],
    layout=Layout(
    display="flex",
    flex_flow='column',
    width="100%",
    height="100%",
    margin="60px 0",
    justify_content='center',
    align_items="center",))



wrapper_container = widgets.Box([header,main_container,home_view_container],
                               layout=Layout(display="flex",flex_flow="column"))
main_container.layout.display = 'none'
display(wrapper_container)


def build_experiment_clicked(b): 
    global new_experiment
    new_experiment = True
    error_output.clear_output()
    with error_output:
        start_server()

    
def connect_to_database_clicked(b):
    error_output.clear_output()
    with error_output:
        resume_server()
        
 
    
build_experiment.on_click(build_experiment_clicked)





Output(layout=Layout(border='2px solid blue'))

Box(children=(HTML(value='\n<div class="nav" style=" \nmargin: 0px;\nbackground-color: #F2F2F2;\nfont-weight: …